In [1]:
#Importing Necessary Libraries
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance
import shutil

In [8]:
train_frame_folder = 'Dataset/train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)

In [9]:
for key, value in data.items():
    print(data[key])

{'label': 'FAKE', 'split': 'train', 'original': 'vudstovrck.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'jdubbvfswz.mp4'}
{'label': 'REAL', 'split': 'train', 'original': None}
{'label': 'FAKE', 'split': 'train', 'original': 'atvmxvwyns.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'kbvibjhfzo.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'ccfoszqabv.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'fjlyaizcwc.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'ffcwhpnpuw.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'slwkmefgde.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'fysyrqfguw.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'qjlhemtkxk.mp4'}
{'label': 'FAKE', 'split': 'train', 'original': 'dlpoieqvfb.mp4'}
{'label': 'REAL', 'split': 'train', 'original': None}
{'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}
{'label': 'FAKE', 'split': 'train'

In [38]:
fake_count=0
real_count=0

for key, value in data.items():
    if "FAKE" == data[key]['label']:
        fake_count+=1
    if "REAL" == data[key]['label']:
        real_count+=1
print("Real Videoos:: ", real_count)
print("Fake Videos:: ", fake_count)

Real Videoos::  77
Fake Videos::  323


In [1]:
import os
import cv2
import json
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import dlib

# Constants
IMG_SIZE = (299, 299)  # Resize detected faces to this size
MOTION_THRESHOLD = 20  # Lowered threshold for detecting motion between frames
FRAME_SKIP = 2        # Reduced frame skip to ensure more frames are processed

# Initialize Dlib's frontal face detector (can switch to Haar cascades, MTCNN, or other)
detector = dlib.get_frontal_face_detector()

def extract_faces_from_frame(frame, detector):
    """
    Detects faces in a frame and returns the resized faces.

    Parameters:
    - frame: The video frame to process.
    - detector: Dlib face detector.

    Returns:
    - resized_faces (list): List of resized faces detected in the frame.
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_frame)
    resized_faces = []

    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        crop_img = frame[y1:y2, x1:x2]
        if crop_img.size != 0:  # Ensure cropped image is valid
            resized_face = cv2.resize(crop_img, IMG_SIZE)
            resized_faces.append(resized_face)

    # Debug: Log the number of faces detected
    #print(f"Detected {len(resized_faces)} faces in current frame")
    return resized_faces

def process_frame(video_path, detector, frame_skip):
    """
    Processes frames to extract motion and face data concurrently.

    Parameters:
    - cap: OpenCV VideoCapture object.
    - detector: Dlib face detector.
    - frame_skip (int): Number of frames to skip for processing.

    Returns:
    - motion_frames (list): List of motion-based face images.
    - all_faces (list): List of all detected faces for fallback.
    """
    prev_frame = None
    frame_count = 0
    motion_frames = []
    all_faces = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames to improve processing speed
        if frame_count % frame_skip != 0:
            frame_count += 1
            continue

        # Debug: Log frame number being processed
        #print(f"Processing frame {frame_count}")

        # # Resize frame to reduce processing time (optional, adjust size as needed)
        # frame = cv2.resize(frame, (640, 360))

        # Extract faces from the current frame
        faces = extract_faces_from_frame(frame, detector)
        all_faces.extend(faces)  # Store all faces detected, including non-motion

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is None:
            prev_frame = gray_frame
            frame_count += 1
            continue

        # Calculate frame difference to detect motion
        frame_diff = cv2.absdiff(prev_frame, gray_frame)
        motion_score = np.sum(frame_diff)

        # Debug: Log the motion score
        #print(f"Motion score: {motion_score}")

        # Check if motion is above the defined threshold and add the face to motion frames
        if motion_score > MOTION_THRESHOLD and faces:
            motion_frames.extend(faces)

        prev_frame = gray_frame
        frame_count += 1

    cap.release()
    return motion_frames, all_faces

def select_well_distributed_frames(motion_frames, all_faces, no_of_frames):
    """
    Selects well-distributed frames from the detected motion and fallback faces.

    Parameters:
    - motion_frames (list): List of frames with detected motion.
    - all_faces (list): List of all detected faces.
    - no_of_frames (int): Required number of frames.

    Returns:
    - final_frames (list): List of selected frames.
    """
    # Case 1: Motion frames exceed the required number
    if len(motion_frames) >= no_of_frames:
        interval = len(motion_frames) // no_of_frames
        distributed_motion_frames = [motion_frames[i * interval] for i in range(no_of_frames)]
        return distributed_motion_frames

    # Case 2: Motion frames are less than the required number
    needed_frames = no_of_frames - len(motion_frames)

    # If all frames together are still less than needed, return all frames available
    if len(motion_frames) + len(all_faces) < no_of_frames:
        #print(f"Returning all available frames: {len(motion_frames) + len(all_faces)}")
        return motion_frames + all_faces

    interval = max(1, len(all_faces) // needed_frames)
    additional_faces = [all_faces[i * interval] for i in range(needed_frames)]

    combined_frames = motion_frames + additional_faces
    interval = max(1, len(combined_frames) // no_of_frames)
    final_frames = [combined_frames[i * interval] for i in range(no_of_frames)]

    return final_frames

def create_dataset(no_of_frames, train_frame_folder):
    """
    Create dataset from DFDC videos by extracting frames based on motion and saving detected faces.

    Parameters:
    - no_of_frames (int): Number of frames to save per video.
    - train_frame_folder (str): Directory containing video files and metadata.json.
    """
    fake_count = 0
    videoprocessedcount = 0

    # Load metadata for video labels
    with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
        data = json.load(file)

    list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]

    with ThreadPoolExecutor(max_workers=16) as executor:
        for vid in list_of_train_data:
            if data[vid]['label'] == "FAKE" and fake_count >= 77:
                continue
            
            video_path = os.path.join(train_frame_folder, vid)

            save_dir = 'Dataset/real' if data[vid]['label'] == 'REAL' else 'Dataset/fake'

            # Extract motion frames and all faces concurrently
            motion_frames, all_faces = process_frame(video_path, detector, FRAME_SKIP)

            # Select well-distributed frames based on the motion and fallback frames
            final_frames = select_well_distributed_frames(motion_frames, all_faces, no_of_frames)

            if data[vid]['label'] == "FAKE" and len(final_frames)!=0:
                fake_count += 1

            # Save the extracted faces
            for idx, face in enumerate(final_frames):
                filename = os.path.join(save_dir, f'{vid.split(".")[0]}_{idx}.png')
                executor.submit(cv2.imwrite, filename, face)  # Save images concurrently

            videoprocessedcount += 1
            print(f"Done processing video {videoprocessedcount}: {vid}, saved {len(final_frames)} frames")

# Example usage
no_of_frames = 10  # Number of frames to extract per video
train_frame_folder = 'Dataset/train_sample_videos'  # Replace with the actual path to the training folder
create_dataset(no_of_frames, train_frame_folder)


Done processing video 1: aagfhgtpmv.mp4, saved 10 frames
Done processing video 2: aapnvogymq.mp4, saved 10 frames
Done processing video 3: abarnvbtwb.mp4, saved 10 frames
Done processing video 4: abofeumbvv.mp4, saved 0 frames
Done processing video 5: abqwwspghj.mp4, saved 10 frames
Done processing video 6: acifjvzvpm.mp4, saved 10 frames
Done processing video 7: acqfdwsrhi.mp4, saved 10 frames
Done processing video 8: acxnxvbsxk.mp4, saved 10 frames
Done processing video 9: acxwigylke.mp4, saved 10 frames
Done processing video 10: aczrgyricp.mp4, saved 10 frames
Done processing video 11: adhsbajydo.mp4, saved 0 frames
Done processing video 12: adohikbdaz.mp4, saved 10 frames
Done processing video 13: adylbeequz.mp4, saved 10 frames
Done processing video 14: aelfnikyqj.mp4, saved 10 frames
Done processing video 15: aelzhcnwgf.mp4, saved 10 frames
Done processing video 16: aettqgevhz.mp4, saved 10 frames
Done processing video 17: aevrfsexku.mp4, saved 10 frames
Done processing video 18:

In [2]:
import os

# Check if the directory exists
fake_images_directory = 'Dataset/fake'

if os.path.exists(fake_images_directory):
    # Count the number of image files in the directory
    fake_images_count = len([file for file in os.listdir(fake_images_directory) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))])
    print(f"The number of images in the 'fake' directory: {fake_images_count}")
else:
    print("The directory does not exist. Please check the path.")

The number of images in the 'fake' directory: 770


In [3]:
import os

# Check if the directory exists
fake_images_directory = 'Dataset/real'

if os.path.exists(fake_images_directory):
    # Count the number of image files in the directory
    fake_images_count = len([file for file in os.listdir(fake_images_directory) if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))])
    print(f"The number of images in the 'real' directory: {fake_images_count}")
else:
    print("The directory does not exist. Please check the path.")

The number of images in the 'real' directory: 760
